# TensorFlowでのモデルサイズ推定

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

### GPUメモリ確保モードの変更

既定では、利用可能なメモリを一括確保する設定であるため、これを必要の都度確保する設定に変更している。

https://www.tensorflow.org/guide/gpu

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## 開始前のGPUメモリ

In [3]:
mem_info_before = tf.config.experimental.get_memory_info('GPU:0')
mem_info_before

{'current': 0, 'peak': 0}

In [4]:
!nvidia-smi

Fri Apr 15 07:20:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W /  70W |    264MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 混合精度演算設定

https://www.tensorflow.org/guide/mixed_precision

In [5]:
dtype = "mixed_float16"
policy = tf.keras.mixed_precision.experimental.Policy(dtype)
tf.compat.v2.keras.mixed_precision.experimental.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


## モデルサイズ計算関数

下記のページのプログラム
https://stackoverflow.com/questions/43137288/how-to-determine-needed-memory-of-keras-model
では、混合精度演算には対応していないため、モデル中の各レイヤのdtype_policyを元にデータサイズを推定するように修正を試みた。

In [6]:
def get_model_memory_usage_mixed_mode(batch_size, model):
    import numpy as np
    try:
        from keras import backend as K
    except:
        from tensorflow.keras import backend as K

    def get_bytes(dtype_policy_name):
        if dtype_policy_name == 'float16':
            return 2
        elif dtype_policy_name == 'float32':
            return 4
        elif dtype_policy_name == 'float64':
            return 8
        if dtype_policy_name == 'mixed_float16':
            return 2
        if dtype_policy_name == 'mixed_bfloat16':
            return 2
        else:
            raise "unknow dtype_plocy;{}".format(dtype_policy_name)
    
    internal_model_mem_count = 0
    trainable_count = 0
    non_trainable_count = 0
    total_memory = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage_mixed_mode(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
 
        trainable_count = np.sum([K.count_params(p) for p in l.trainable_weights])
        non_trainable_count = np.sum([K.count_params(p) for p in l.non_trainable_weights])

        total_memory += get_bytes(l.dtype_policy.name) * batch_size * single_layer_mem
        total_memory += get_bytes(l.dtype_policy.name) * (trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes

## 3D CNNの場合
下記のページより  
https://keras.io/examples/vision/3D_image_classification/

In [7]:
NUM_CLASSES = 10

In [8]:
def get_model(width=128, height=128, depth=64, channel=1):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, channel))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(units=NUM_CLASSES)(x)
    outputs = layers.Activation(keras.activations.softmax, dtype='float32')(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=150, height=300, depth=150, channel=3)
model.summary()

Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 300, 150, 3  0         
                             )]                                  
                                                                 
 conv3d (Conv3D)             (None, 148, 298, 148, 64  5248      
                             )                                   
                                                                 
 max_pooling3d (MaxPooling3D  (None, 74, 149, 74, 64)  0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 74, 149, 74, 64)  256       
 ormalization)                                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 72, 147, 72, 64)   110656

In [9]:
for batch_size in [2, 4, 8, 16, 32]:
  mem_size = get_model_memory_usage_mixed_mode(batch_size, model)
  print("Batch Size:{} , Memory Usage: {}GB".format(batch_size, mem_size))

Batch Size:2 , Memory Usage: 2.383GB
Batch Size:4 , Memory Usage: 4.763GB
Batch Size:8 , Memory Usage: 9.524GB
Batch Size:16 , Memory Usage: 19.045GB
Batch Size:32 , Memory Usage: 38.088GB


## ダミーデータによるモデル訓練

In [10]:
BATCH_SIZE = 2
dummy_x = np.random.randn(BATCH_SIZE, 150, 300, 150, 3)
dummy_y = np.random.randint(0, 9, size=BATCH_SIZE)

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(dummy_x, dummy_y)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)

1/1 [==============================] - 15s 15s/step - loss: 2.2639


## 訓練実行後のGPUメモリ

In [12]:
mem_info_after = tf.config.experimental.get_memory_info('GPU:0')
mem_info_after

{'current': 287205120, 'peak': 7991122432}

In [13]:
peak_allocated_mem_gb = (mem_info_after['peak'] - mem_info_before['peak']) / 1024**3
peak_allocated_mem_gb

7.442312717437744

推定モデルサイズ2.38GBに対して3倍以上のメモリが確保されている。

In [14]:
!nvidia-smi

Fri Apr 15 07:20:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |  10036MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

実際には10GB近いメモリが占有されている。

## 考察

### (1) 上記のモデルサイズ推定プログラムの問題点

上記の推定プログラムでは、レイヤごとの訓練可能パラメータ（重み）と訓練不可パラメータおよび出力テンソルのサイズを合計して、モデルの使用メモリサイズを推定している。

これは推論時を想定した計算であり、訓練時のバックプロパゲーションの勾配計算に必要なメモリを想定していない。

また、次の実験が示すように、実質的に同じ計算グラフを生成するモデルでも、記述方法が異なると計算が異なってしまうという問題を抱えている。

#### Activationを別レイヤにした場合の計算

In [15]:
def get_model(width=128, height=128, depth=64, channel=1):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, channel))

    x = layers.Conv3D(filters=64, kernel_size=3)(inputs)
    x = layers.Activation(tf.keras.activations.relu)(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3)(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3)(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3)(x)
    x = layers.Activation(tf.keras.activations.relu)(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(units=10)(x)
    outputs = layers.Activation(tf.keras.activations.softmax)(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn-2")
    return model


# Build model.
model = get_model(width=150, height=300, depth=150, channel=3)
model.summary()

Model: "3dcnn-2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 300, 150, 3  0         
                             )]                                  
                                                                 
 conv3d_4 (Conv3D)           (None, 148, 298, 148, 64  5248      
                             )                                   
                                                                 
 activation_1 (Activation)   (None, 148, 298, 148, 64  0         
                             )                                   
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 74, 149, 74, 64)  0         
 3D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 74, 149, 74, 64)  256 

In [16]:
for batch_size in [2, 4, 8, 16, 32]:
  mem_size = get_model_memory_usage_mixed_mode(batch_size, model)
  print("Batch Size:{} , Memory Usage: {}GB".format(batch_size, mem_size))

Batch Size:2 , Memory Usage: 4.167GB
Batch Size:4 , Memory Usage: 8.332GB
Batch Size:8 , Memory Usage: 16.661GB
Batch Size:16 , Memory Usage: 33.319GB
Batch Size:32 , Memory Usage: 66.635GB


上記のように、Activationをレイヤとして定義すると、メモリ推定量が増加する。

### (2)TensorFlowから見たメモリアロケーションとnvidia-smiでの数値の差

TensorFlowから見たget_memory_infoで得られるメモリアロケーションのピーク値とnvidia-smiで確認したメモリ占有量が異なり、後者が大きくなっている理由として考えられるのは、リリース後のメモリが再利用されずメモリフラグメンテーションが起きている可能性がある。

### (3)精度の高いメモリサイズ推定法について

単にバックプロパゲーション時の勾配計算用のメモリだけであれば、これまでの計算のメモリサイズを2倍にすれば大きな差はないはずであるが、実際に確保されたメモリーを見ると3倍から4倍のメモリーが必要と考えられる。

深層学習モデルの学習に必要なメモリサイズに関する研究では、Microsoftが下記の論文を発表している。

https://www.microsoft.com/en-us/research/uploads/prod/2020/09/dnnmem.pdf

この論文では、TensorFlow、PyTorch、MXNetについて、その計算グラフやモデルのソースコード、訓練用パラメータから必要メモリを推定するアルゴリズムを紹介しているが、残念ながら計算プログラムDNNMemについては公開されていない。